In [1]:
#imports
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection, metrics
from imblearn.over_sampling import SMOTE
import csv

In [2]:
# Util functions
def classifierEstimation(classifier, x_test, y_test):
    print
    print('Test set has ', len([i for i in y_test if i is 1]), 'positives and ', len([i for i in y_test if i is 0]), ' negatives')
    print
    print ('Best classifier score')
    print
    print(metrics.classification_report(y_test, classifier.predict(x_test), target_names=['non-binding', 'binding']))

In [8]:
#import data

positives_dna = []
negatives_dna = []
positives_enzyme = []
negatives_enzyme = []

with open('data/minimal_model_pdidb_positives.txt') as p_dna:
    for row in csv.reader(p_dna,delimiter="\t"):
        positives_dna.append(row)

with open('data/minimal_model_pdidb_negatives.txt') as n_dna:
    for row in csv.reader(n_dna,delimiter="\t"):
        negatives_dna.append(row)
       
with open('data/minimal_model_enzyme_positives.txt') as p_enzyme:
    for row in csv.reader(p_enzyme,delimiter="\t"):
        positives_enzyme.append(row)
        
with open('data/minimal_model_enzyme_negatives.txt') as n_enzyme:
    for row in csv.reader(n_enzyme,delimiter="\t"):
        negatives_enzyme.append(row)

In [9]:
#preprocess data

tmp_dna = positives_dna + negatives_dna
tmp_enzyme = positives_enzyme + negatives_enzyme

labels_dna = [1]*len(positives_dna)+[0]*len(negatives_dna)
labels_enzyme = [1]*len(positives_enzyme)+[0]*len(negatives_enzyme)

dna = []
enzyme = []

for j in tmp_dna:
    new_j = [float(i) for i in j]
    dna.append(new_j)
    
for j in tmp_enzyme:
    new_j = [float(i) for i in j]
    enzyme.append(new_j)

In [10]:
#train and test datasets

dna_x_train,dna_x_test,dna_y_train,dna_y_test = model_selection.train_test_split(dna, labels_dna, test_size=0.2, random_state=6)
enzyme_x_train,enzyme_x_test,enzyme_y_train,enzyme_y_test = model_selection.train_test_split(enzyme,labels_enzyme,test_size=0.2,random_state=6)

x_train = dna_x_train + enzyme_x_train
y_train = dna_y_train + enzyme_y_train
x_test = dna_x_test + enzyme_x_test
y_test = dna_y_test + enzyme_y_test

sm = SMOTE(random_state=42)
x_train, y_train = sm.fit_sample(x_train,y_train)

kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=3)
parameters = {'alpha': [0.0001], 'random_state': [1]}

In [11]:
#training the neural network

grid = model_selection.GridSearchCV(MLPClassifier(), parameters, cv=kf)
grid.fit(x_train,y_train)
classifier = grid.best_estimator_

In [12]:
#testing

#prediction = classifier.predict(enzyme_x_test)
prediction = classifier.predict(x_test)

unique, counts = np.unique(prediction, return_counts=True)
print(dict(zip(unique, counts)))

#print('Prediction has positive', len([i for i in prediction if i is 1]), 'and negative', len([i for i in prediction if i is 0]))

classifierEstimation(classifier,enzyme_x_test,enzyme_y_test)
classifierEstimation(classifier,dna_x_test, dna_y_test)
classifierEstimation(classifier,x_test,y_test)

print("Training set score: %f" % classifier.score(x_train, y_train))
print("Test set score: %f" % classifier.score(x_test, y_test))
print(classifier.n_layers_)
print(classifier.n_iter_)

{0: 10728, 1: 4905}
Test set has  1335 positives and  11615  negatives
Best classifier score
             precision    recall  f1-score   support

non-binding       0.93      0.72      0.82     11615
    binding       0.19      0.56      0.28      1335

avg / total       0.86      0.71      0.76     12950

Test set has  363 positives and  2320  negatives
Best classifier score
             precision    recall  f1-score   support

non-binding       0.92      0.69      0.79      2320
    binding       0.24      0.62      0.34       363

avg / total       0.83      0.68      0.73      2683

Test set has  1698 positives and  13935  negatives
Best classifier score
             precision    recall  f1-score   support

non-binding       0.93      0.72      0.81     13935
    binding       0.20      0.57      0.29      1698

avg / total       0.85      0.70      0.75     15633



Training set score: 0.670704
Test set score: 0.701337
3
107
